In [1]:
import json
from data.dataset import NERDataset
from models.networks import GlobalContextualDeepTransition
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
sourceName = 'data/conll03/eng.train.src'
targetName = 'data/conll03/eng.train.trg'
gloveFile = 'data/conll03/trimmed.300d.Cased.txt'
symbFile = 'data/conll03/sym.glove'
data = NERDataset(sourceName, targetName, gloveFile, symbFile)
loader = data.getLoader(4096)

In [3]:
with open('config.json', 'r') as file:
    kwargs = json.load(file)
model = GlobalContextualDeepTransition(**kwargs)

In [4]:
# add glove weights here
model.init_weights(data.embeddingWeights)

numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {numParams:,}")

Trainable parameters: 7,313,341


In [5]:
ckpt = ModelCheckpoint(filepath='lightning_logs/checkpoint', period=1)

In [6]:
trainer = pl.Trainer(#resume_from_checkpoint='lightning_logs/checkpoint-v0.ckpt',
                    gradient_clip_val=5., gpus=1,
                     max_epochs=2, checkpoint_callback=ckpt)
trainer.fit(model, loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                           | Params
--------------------------------------------------------------------
0 | contextEncoder   | GlobalContextualEncoder        | 7 M   
1 | sequenceLabeller | SequenceLabelingEncoderDecoder | 6 M   


c:\users\deepak h r\appdata\local\programs\python\python37\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
c:\users\deepak h r\appdata\local\programs\python\python37\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


1